# [초급 프로젝트] 4팀_김명환 - 알약객체 증강

---
---

# 환경설정
    - 라이브러리 설치 및 로딩
    - 사용자 함수 사용

In [ ]:
!pip install -q gdown
!pip install -q albumentations
!pip install -q ultralytics
!pip install -q -U ultralytics
!pip install -q nbformat
!pip install -q roboflow
!pip install -q opencv-python
!pip install -q opencv-python-headless
!pip install -q wandb
!pip install -q timm
!pip install -q torchvision
#!pip install -q torch torchvision tqdm pillow matplotlib

print("로딩완료")

In [ ]:
# 기본 라이브러리 (중복 제거 및 정리)

# --- Scikit-learn: 데이터 전처리, 모델, 평가 ---
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import (
    fetch_california_housing, load_iris, make_moons, make_circles,
    load_breast_cancer, load_wine
)
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, average_precision_score

# --- 이미지 처리 ---
import cv2
from PIL import Image, ImageFilter, ImageDraw
import albumentations as A

# --- PyTorch: 딥러닝 관련 ---
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, Subset
# 문제 있는 v2 import 제거하고 필요시에만 개별적으로 import
# from torchvision.transforms import v2, functional as TF
from torchvision.transforms import functional as TF
from torchvision.datasets import CocoDetection
from torch.nn import CrossEntropyLoss
from collections import OrderedDict

# --- COCO 데이터셋 관련 ---
from pycocotools.coco import COCO
from pycocotools import mask as coco_mask

# --- 딥러닝 모델 ---
import timm

# --- 기본 라이브러리 ---
import os
import sys
import re
import csv
import copy
import json
import math
import random
import yaml
import shutil
import requests
import xml.etree.ElementTree as ET
from pathlib import Path

# --- 데이터 분석 및 시각화 ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# --- 시간 관련 ---
from datetime import datetime, timezone, timedelta
import pytz

# --- 진행률 표시 ---
import IPython.display
from tqdm.notebook import tqdm

# --- 시간대 설정 ---
__kst = pytz.timezone('Asia/Seoul')

# --- GPU 설정 ---
__device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
__device_cpu = torch.device('cpu')

# --- 재현 가능한 결과를 위한 시드 설정 ---
np.random.seed(42)
torch.manual_seed(42)
if __device.type == 'cuda':
    torch.cuda.manual_seed_all(42)

print(f"라이브러리 로드 완료 사용장치: {__device}")

In [ ]:
# OS 및 경로 관련 라이브러리 임포트
import os, sys
from pathlib import Path

try:
    import google.colab
    from google.colab import drive
    COLAB_AVAILABLE = True
except ImportError:
    COLAB_AVAILABLE = False

# 유틸리티 함수 디렉토리 경로 설정 (Colab 환경 여부에 따라 다르게 지정)
utils_dir = None
if COLAB_AVAILABLE:
    # Colab 환경일 경우 Google Drive 경로 사용
    utils_dir = "/content/drive/MyDrive/codeit_ai_health_eat/src/python_modules/utils"
else:
    # 로컬 환경일 경우 현재 드라이브 기준 경로 사용
    utils_dir = os.path.join(Path.cwd().drive + '\\\\', 'GoogleDrive', "codeit_ai_health_eat", "src", "python_modules", "utils")

print("utils_dir:", utils_dir)

# 유틸리티 함수 경로를 파이썬 모듈 검색 경로에 추가
sys.path.append(str(utils_dir))
print("sys.path:", sys.path)

# health_ea_utils 모듈 임포트 및 최신화(reload)
import importlib
import health_ea_utils as heu
importlib.reload(heu)
from health_ea_utils import *

# helper 및 health_ea_utils 파일 경로 출력 (디버깅용)
print("helper.__file__:", helper.__file__)
print("health_ea_utils.__file__:", heu.__file__)

In [ ]:
import importlib
import helper_utils as hutils
importlib.reload(hutils)
from helper_utils import *


# 데이타 다운로드

In [ ]:
# download_files 변수 예시 (Google Drive 및 Naver MyBox 링크)
# 여러 버전의 데이터셋 다운로드 링크를 주석으로 관리

# download_files={
#     'yolo_label_one_class' : r'https://drive.google.com/file/d/177_86k4BuT6JnFnq7ZHJtEjp7jaRbCl2/view?usp=sharing',
#     'yolo_label' : r'https://drive.google.com/file/d/1nc-WFcw7lCS7s7VGzN9Kxh80PiBBggez/view?usp=sharing',
#     'yolo_resize_one_class' : r'https://drive.google.com/file/d/1Ak0EvkMnuwvcAFvTO-zovIgVcNlROjsS/view?usp=sharing',
#     'yolo_resize' : r'https://drive.google.com/file/d/1kpo57qOJhEhrkuzUCEh57ILB5xSPVoFv/view?usp=sharing',
# }

# download_files={
#     'yolo_label' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODU5OTkzNTMyOHxGfDA&svcType=MYBOX-WEB&time=1757776010785',
#     'yolo_label_one_class' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODYzOTg5NDExMnxGfDA&svcType=MYBOX-WEB&time=1757776673721',
#     'yolo_resize_one_class' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODgwNjk2NDMyMHxGfDA&svcType=MYBOX-WEB&time=1757780142635',
#     'yolo_resize' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODY4MDc2MjQ2NHxGfDA&svcType=MYBOX-WEB&time=1757780177672',
# }

# 실제로 사용할 데이터셋 다운로드 링크만 활성화
download_files={
    # 'yolo_label' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODU5OTkzNTMyOHxGfDA&svcType=MYBOX-WEB&time=1757776010785',
    # 'yolo_label_one_class' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODYzOTg5NDExMnxGfDA&svcType=MYBOX-WEB&time=1757776673721',
    # 'yolo_resize_one_class' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODgwNjk2NDMyMHxGfDA&svcType=MYBOX-WEB&time=1757780142635',
    # 'yolo_resize' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODY4MDc2MjQ2NHxGfDA&svcType=MYBOX-WEB&time=1757780177672',
    'yolo_noresize' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc2NDA0ODY2ODI1NnxGfDA&svcType=MYBOX-WEB&time=1757851996107',
    #'yolo_noresize_one_class':r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc2NDgwMDcxODk0NHxGfDA&svcType=MYBOX-WEB&time=1757893856220',
}

# 참고용: 각 데이터셋의 직접 다운로드 링크
# yolo_noresize = https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc2NDA0ODY2ODI1NnxGfDA&svcType=MYBOX-WEB&time=1757851996107
# yolo_noresize_one_class = https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc2NDgwMDcxODk0NHxGfDA&svcType=MYBOX-WEB&time=1757893856220


In [ ]:
import gdown
def download_gdrive_file(url, output_path, ignore=True):
    """Google Drive 파일 다운로드 함수

    Args:
        url (str): Google Drive 공유 링크
        output_path (str): 다운로드할 파일 경로
        ignore (bool, optional): True면 기존 파일 삭제 후 다운로드, False면 파일 있으면 건너뜀. Defaults to True.

    Raises:
        ValueError: Google Drive 파일 ID를 찾을 수 없습니다.
    """
    # 공유 링크에서 파일 ID 추출
    if os.path.exists(output_path):
        if ignore:
            os.remove(output_path)
        else:
            return

    file_id_match = re.search(r'/d/([a-zA-Z0-9_-]+)', url)
    if not file_id_match:
        raise ValueError("Google Drive 파일 ID를 찾을 수 없습니다.")
    file_id = file_id_match.group(1)
    gdown.download(f"https://drive.google.com/uc?id={file_id}", output_path, quiet=False)

def download_http(url, target, ignore=True):
    """
    HTTP 파일 다운로드 함수 (진행률 표시)
    url: 다운로드할 파일 URL
    target: 저장할 파일 경로
    ignore: True면 기존 파일 삭제 후 다운로드, False면 파일 있으면 건너뜀
    """
    if os.path.exists(target):
        if ignore:
            os.remove(target)
        else:
            print(f"이미 파일이 존재합니다: {target}")
            return target

    response = requests.get(url, stream=True)
    total = int(response.headers.get('content-length', 0))
    with open(target, 'wb') as file, tqdm(
        desc=f"Downloading {os.path.basename(target)}",
        total=total,
        unit='B',
        unit_scale=True,
        unit_divisor=1024,
        ascii=True
    ) as bar:
        for data in response.iter_content(chunk_size=1024):
            size = file.write(data)
            bar.update(size)
    print(f"다운로드 완료: {target}")
    return target

# local_code_it_ai04 = os.path.join( '~/.cache/' if helper.is_colab else Path.cwd().drive + '\\'
#                                   ,'temp'
#                                   , 'code_it_ai04')

# 데이터 다운로드 및 압축 해제 코드에 상세 주석 추가

# 다운로드 경로 설정 (Colab/로컬 환경에 따라 다름)
if COLAB_AVAILABLE:
    local_code_it_ai04 = os.path.join( '/content/', 'code_it_ai04')
else:
    local_code_it_ai04 = os.path.join( Path.cwd().drive + '\\\\', 'temp', 'code_it_ai04')

print("local_code_it_ai04:", local_code_it_ai04)

os.makedirs(local_code_it_ai04, exist_ok=True)  # 폴더 생성 코드 추가

unzip_paths = []
for key, url in download_files.items():
    print(f"{key}: {url}")
    zipfile = os.path.join(local_code_it_ai04, f'{key}.zip')
    unzip_path = os.path.join(local_code_it_ai04, f'{key}.zip.unzip')
    # 이미 압축해제된 폴더가 있으면 재다운로드/압축해제하지 않음
    if os.path.exists(unzip_path):
        print(f"이미 압축해제된 폴더가 존재합니다: {unzip_path}")
        print('unzipfile:', unzip_path)
        unzip_paths.append(unzip_path)
        continue
    # Google Drive 파일 다운로드 함수 (주석처리, 필요시 사용)
    #download_gdrive_file(url, os.path.join(local_code_it_ai04, f'{key}.zip'), ignore=False)
    # 일반 HTTP 다운로드 함수 사용
    download_http(url, zipfile, ignore=False)
    # 압축 해제 (health_ea_utils의 unzip 함수 사용)
    unzip_path_list = heu.unzip([os.path.join(local_code_it_ai04, f'{key}.zip')])
    # 압축 해제된 경로 리스트 출력 및 저장
    print('unzip_path_list:', unzip_path_list)
    unzip_paths.extend(unzip_path_list)


In [ ]:
# google drive root에 keggle.json 파일 필요합니다.
for path in unzip_paths:
    print("압축해제된 폴더:", path)

#yolo_dataset_path = os.path.join(local_code_it_ai04, f'yolo_label_one_class.zip.unzip')
yolo_dataset_path =unzip_paths[0]
yaml_path = os.path.join(yolo_dataset_path, "dataset.yaml")

def get_path_data():
    path = yolo_dataset_path
    return path

print("yaml_path:", yaml_path)
print("get_path_data:", get_path_data())

In [ ]:
class YOLOToClassificationDataset(Dataset):
    """
    YOLO 형식의 객체 감지 데이터셋을 이미지 분류용 데이터셋으로 변환하는 클래스입니다.

    - yaml_path: YOLO 데이터셋의 dataset.yaml 파일 경로
    - split: 'train', 'val', 'test' 중 하나로 데이터 분할 선택
    - transform: 이미지 전처리(transform) 함수 (torchvision.transforms 등)
    - crop_objects: True일 경우, 바운딩 박스(bbox) 영역만 crop하여 분류 이미지로 사용

    주요 동작:
    1. yaml 파일에서 이미지/라벨 폴더 경로, 클래스 정보 등을 읽어옴
    2. 각 라벨(txt) 파일을 읽어, 이미지 경로와 클래스, bbox 정보를 self.data에 저장
    3. __getitem__에서 bbox 영역 crop 후 transform 적용, (이미지, 클래스ID) 반환
    4. get_item은 추가로 원본 이미지 경로와 bbox 좌표도 반환 (샘플 저장 등에 활용)

    예시 사용법:
        dataset = YOLOToClassificationDataset(yaml_path, split='train', transform=...)
        image, label = dataset[0]
    """


    def __init__(self, yaml_path, split='train', transform=None, crop_objects=True):
        """YOLOToClassificationDataset 초기화

        Args:
            yaml_path (str): YOLO 데이터셋의 dataset.yaml 파일 경로
            split (str, optional): 'train', 'val', 'test' 중 하나로 데이터 분할 선택. Defaults to 'train'.
            transform (callable, optional): 이미지 전처리(transform) 함수 (torchvision.transforms 등). Defaults to None.
            crop_objects (bool, optional): True일 경우, 바운딩 박스(bbox) 영역만 crop하여 분류 이미지로 사용. Defaults to True.

        Raises:
            ValueError: 잘못된 split 값이 주어진 경우
        """
        # yaml 파일 읽기
        with open(yaml_path, 'r') as f:
            yaml_data = yaml.safe_load(f)

        self.yaml_data_path = yaml_data['path']
        self.nc = yaml_data['nc']
        self.names = yaml_data['names']
        self.yaml_data_train = os.path.join(yaml_data['path'], yaml_data['train'])
        self.yaml_data_val = os.path.join(yaml_data['path'], yaml_data['val'])
        self.yaml_data_test = os.path.join(yaml_data['path'], yaml_data['test'])

        print("yaml_data_path:", self.yaml_data_path)

        # split에 따라 경로 선택
        if split == 'train':
            image_dir = self.yaml_data_train
        elif split == 'val':
            image_dir = self.yaml_data_val
        elif split == 'test':
            image_dir = self.yaml_data_test
        else:
            raise ValueError(f"split 값이 잘못되었습니다: {split}")

        label_dir = image_dir.replace('images', 'labels')
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.class_names = self.names
        self.transform = transform
        self.crop_objects = crop_objects
        self.data = []

        self._load_data()

    def _load_data(self):
        """
        데이터셋을 로드하는 함수

        - 라벨 디렉토리(self.label_dir) 내의 모든 .txt 파일(객체 감지 라벨)을 순회합니다.
        - 각 라벨 파일의 한 줄(line)은 하나의 객체(알약)에 대한 정보입니다.
        - 각 줄에서 class_id, bbox(x_center, y_center, width, height)를 읽어옵니다.
        - 해당 라벨 파일에 대응하는 이미지 파일(.jpg)을 찾고, 존재하면
        이미지 경로, 클래스ID, 바운딩박스 정보를 self.data 리스트에 저장합니다.
        - 즉, 한 알약(객체)마다 하나의 데이터 샘플이 만들어집니다.
        - 이후 __getitem__에서 bbox 영역만 crop(클리핑)하여 분류용 이미지로 반환합니다.

        반환값: 없음 (self.data에 샘플 정보가 누적됨)
        """

        if not os.path.exists(self.label_dir):
            #print(f"라벨 폴더가 존재하지 않습니다: {self.label_dir}")
            return  # 라벨 폴더 없으면 빈 데이터셋
        for label_file in os.listdir(self.label_dir):
            if not label_file.endswith('.txt'):
                continue
            image_file = label_file.replace('.txt', '.jpg')
            image_path = os.path.join(self.image_dir, image_file)
            label_path = os.path.join(self.label_dir, label_file)
            if not os.path.exists(image_path):
                continue
            with open(label_path, 'r') as f:
                lines = f.readlines()
            for line in lines:
                parts = line.strip().split()
                if len(parts) >= 5:
                    class_id = int(parts[0])
                    x_center, y_center, width, height = map(float, parts[1:5])
                    self.data.append({
                        'image_path': image_path,
                        'class_id': class_id,
                        'bbox': (x_center, y_center, width, height)
                    })

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        """
        데이터셋에서 특정 인덱스의 샘플을 가져오는 함수

        - self.data에서 idx번째 샘플 정보를 가져옵니다.
        - 해당 이미지 파일을 열고, crop_objects=True일 경우 바운딩 박스(bbox) 영역만 crop(클리핑)합니다.
        - 이미지 전처리(transform)가 지정되어 있으면 적용합니다.
        - 최종적으로 (이미지, 클래스ID)를 반환합니다.

        즉, 한 알약 객체의 crop된 이미지와 클래스 라벨을 반환합니다.
        """

        item = self.data[idx]
        image = Image.open(item['image_path']).convert('RGB')
        if self.crop_objects and 'bbox' in item:
            img_w, img_h = image.size
            x_center, y_center, width, height = item['bbox']
            x1 = int((x_center - width/2) * img_w)
            y1 = int((y_center - height/2) * img_h)
            x2 = int((x_center + width/2) * img_w)
            y2 = int((y_center + height/2) * img_h)
            # 좌표가 올바른지 체크
            if x2 > x1 and y2 > y1 and x1 >= 0 and y1 >= 0 and x2 <= img_w and y2 <= img_h:
                image = image.crop((x1, y1, x2, y2))
            # else: 잘못된 bbox는 crop하지 않음
        if self.transform:
            image = self.transform(image)
        return image, item['class_id']

    def get_item(self, idx):
        """
        데이터셋에서 특정 인덱스의 샘플을 가져오는 함수

        - self.data에서 idx번째 샘플 정보를 가져옵니다.
        - 해당 이미지 파일을 열고, crop_objects=True일 경우 바운딩 박스(bbox) 영역만 crop(클리핑)합니다.
        - 이미지, 클래스ID, 원본 이미지 경로, bbox 좌표([x1, y1, x2, y2])를 반환합니다.
        - 샘플 이미지 저장, 시각화 등에 활용할 수 있습니다.

        즉, 한 알약 객체의 crop된 이미지와 클래스 라벨, 원본 경로, bbox 좌표를 반환합니다.
        """

        item = self.data[idx]
        image = Image.open(item['image_path']).convert('RGB')
        if self.crop_objects and 'bbox' in item:
            img_w, img_h = image.size
            x_center, y_center, width, height = item['bbox']
            x1 = int((x_center - width/2) * img_w)
            y1 = int((y_center - height/2) * img_h)
            x2 = int((x_center + width/2) * img_w)
            y2 = int((y_center + height/2) * img_h)
            # 좌표가 올바른지 체크
            if x2 > x1 and y2 > y1 and x1 >= 0 and y1 >= 0 and x2 <= img_w and y2 <= img_h:
                image = image.crop((x1, y1, x2, y2))
            # else: 잘못된 bbox는 crop하지 않음
        return image, item['class_id'], item['image_path'], [x1, y1, x2, y2]


In [ ]:
def update_yaml_paths_to_absolute(yaml_path):
    """
    dataset.yaml 파일의 상대 경로(path)를 절대 경로로 변환하여 저장하는 함수

    Args:
        yaml_path (str): dataset.yaml 파일의 경로

    Returns:
        dict: 절대 경로로 수정된 yaml 데이터 딕셔너리
    """

    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)

    yaml_dir = os.path.dirname(yaml_path)
    data['path'] = os.path.normpath(os.path.join(yaml_dir, data['path']))
    # for key in ['train', 'val', 'test']:
    #     if key in data and not os.path.isabs(data[key]):
    #         data[key] = os.path.normpath(os.path.join(yaml_dir, data[key]))

    with open(yaml_path, 'w') as f:
        yaml.dump(data, f, allow_unicode=True)

    return data

yaml_data = update_yaml_paths_to_absolute(yaml_path)
print(yaml_data.keys())
yaml_data_path = yaml_data['path']
nc = yaml_data['nc']
names = yaml_data['names']
yaml_data_train = os.path.join(yaml_data['path'], yaml_data['train'])
yaml_data_val = os.path.join(yaml_data['path'], yaml_data['val'])
yaml_data_test = os.path.join(yaml_data['path'], yaml_data['test'])

print("nc:", nc)
print("names:", names)
print("yaml_data_path:", yaml_data_path)
print("yaml_data_train:", yaml_data_train)
print("yaml_data_val:", yaml_data_val)
print("yaml_data_test:", yaml_data_test)

coco_data_root = f"{yaml_data_path}_coco"
print("coco_data_root:", coco_data_root)


In [ ]:
original_train_dataset = YOLOToClassificationDataset(
    yaml_path=yaml_path,
    split='train',
    transform=None
)
original_val_dataset = YOLOToClassificationDataset(
    yaml_path=yaml_path,
    split='val',
    transform=None
)
print(len(original_train_dataset), len(original_val_dataset))

In [ ]:
def convert_yolo_to_coco_format(train_dataset, val_dataset, test_dataset, output_dir, image_format='jpg'):
    """
    YOLO 데이터셋을 COCO 형식으로 변환하여 저장하는 함수
    
    Args:
        train_dataset: YOLOToClassificationDataset (train)
        val_dataset: YOLOToClassificationDataset (val) 
        test_dataset: YOLOToClassificationDataset (test)
        output_dir: 출력 폴더 경로
        image_format: 저장할 이미지 형식 ('jpg' 또는 'png')
    """
    import json
    from datetime import datetime
    
    # 출력 디렉토리 구조 생성
    os.makedirs(output_dir, exist_ok=True)
    
    # COCO 형식 기본 구조
    def create_coco_structure(description, split_name):
        return {
            "info": {
                "description": description,
                "url": "",
                "version": "1.0",
                "year": datetime.now().year,
                "contributor": "YOLO to COCO Converter",
                "date_created": datetime.now().isoformat()
            },
            "licenses": [
                {
                    "id": 1,
                    "name": "Unknown License",
                    "url": ""
                }
            ],
            "images": [],
            "annotations": [],
            "categories": []
        }
    
    # 각 분할별로 처리
    datasets_info = [
        (train_dataset, 'train', 'Training dataset'),
        (val_dataset, 'val', 'Validation dataset'), 
        (test_dataset, 'test', 'Test dataset')
    ]
    
    for dataset, split_name, description in datasets_info:
        if dataset is None or len(dataset) == 0:
            print(f"{split_name} 데이터셋이 비어있거나 None입니다. 건너뜁니다.")
            continue
            
        print(f"{split_name} 데이터셋 변환 중... ({len(dataset)} samples)")
        
        # COCO 구조 초기화
        coco_data = create_coco_structure(description, split_name)
        
        # 이미지 및 주석 디렉토리 생성
        images_dir = os.path.join(output_dir, f'{split_name}2017')
        os.makedirs(images_dir, exist_ok=True)
        
        # 카테고리 정보 추가 (한 번만)
        if hasattr(dataset, 'class_names'):
            class_names = dataset.class_names
        elif hasattr(dataset, 'dataset') and hasattr(dataset.dataset, 'class_names'):
            # ConcatDataset의 경우
            for ds in dataset.datasets:
                if hasattr(ds, 'class_names'):
                    class_names = ds.class_names
                    break
                elif hasattr(ds, 'dataset') and hasattr(ds.dataset, 'class_names'):
                    class_names = ds.dataset.class_names
                    break
        else:
            class_names = [f"class_{i}" for i in range(10)]  # 기본값
            
        for i, class_name in enumerate(class_names):
            coco_data["categories"].append({
                "id": i,
                "name": class_name,
                "supercategory": "pill"
            })
        
        # 이미지별 그룹핑을 위한 딕셔너리
        image_groups = {}
        annotation_id = 1
        
        # 모든 샘플을 순회하며 이미지별로 그룹핑
        print(f"{split_name} 샘플들을 이미지별로 그룹핑 중...")
        pbar = tqdm(range(len(dataset)), desc=f"Processing {split_name}")
        
        for idx in pbar:
            # 데이터셋 타입에 따른 처리
            if hasattr(dataset, 'get_item'):
                # 원본 YOLOToClassificationDataset
                crop_image, class_id, original_image_path, bbox = dataset.get_item(idx)
            elif hasattr(dataset, 'dataset') and hasattr(dataset.dataset, 'get_item'):
                # ConcatDataset 내부의 원본 데이터셋 접근
                # 복잡하므로 원본 이미지 정보 추출
                crop_image, class_id = dataset[idx]
                # 원본 경로는 별도로 처리 필요
                original_image_path = f"unknown_{idx}.jpg"
                bbox = [0, 0, 224, 224]  # 기본값
            else:
                crop_image, class_id = dataset[idx]
                original_image_path = f"sample_{idx}.jpg"
                bbox = [0, 0, 224, 224]
                
            # 이미지 파일명 생성
            base_filename = os.path.basename(original_image_path).replace('.jpg', '').replace('.png', '')
            image_filename = f"{base_filename}_{class_id:03d}_{idx:06d}.{image_format}"
            image_path = os.path.join(images_dir, image_filename)
            
            # crop된 이미지 저장
            if isinstance(crop_image, torch.Tensor):
                # 정규화 해제
                if crop_image.shape[0] == 3:  # (C, H, W)
                    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
                    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
                    crop_image = crop_image * std + mean
                    crop_image = torch.clamp(crop_image, 0, 1)
                crop_image = TF.to_pil_image(crop_image)
            
            # 이미지 저장
            crop_image.save(image_path)
            crop_width, crop_height = crop_image.size
            
            # COCO 이미지 정보 추가
            image_info = {
                "id": idx + 1,
                "width": crop_width,
                "height": crop_height,
                "file_name": image_filename,
                "license": 1,
                "flickr_url": "",
                "coco_url": "",
                "date_captured": datetime.now().isoformat()
            }
            coco_data["images"].append(image_info)
            
            # COCO 주석 정보 추가 (전체 이미지가 해당 클래스)
            annotation_info = {
                "id": annotation_id,
                "image_id": idx + 1,
                "category_id": int(class_names[class_id]),
                "segmentation": [],
                "area": crop_width * crop_height,
                "bbox": [0, 0, crop_width, crop_height],  # 전체 이미지
                "iscrowd": 0
            }
            coco_data["annotations"].append(annotation_info)
            annotation_id += 1
            
            pbar.set_postfix_str(f'{class_names[class_id]} ({image_filename})')
                
        
        # JSON 파일 저장
        json_filename = f"instances_{split_name}2017.json"
        json_path = os.path.join(output_dir, json_filename)
        
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(coco_data, f, indent=2, ensure_ascii=False)
        
        print(f"{split_name} 변환 완료:")
        print(f"이미지: {len(coco_data['images'])}개 → {images_dir}")
        print(f"주석: {json_path}")
        print(f"카테고리: {len(coco_data['categories'])}개")


In [ ]:
coco_data_org_root = f"{coco_data_root}_org"
print("coco_data_org_root:", coco_data_org_root)

# 기존 폴더가 있으면 삭제 후 다시 생성
if os.path.exists(coco_data_org_root):
    import shutil
    shutil.rmtree(coco_data_org_root)
    
if not os.path.exists(coco_data_org_root):
    os.makedirs(coco_data_org_root)
    # COCO 형식으로 변환
    convert_yolo_to_coco_format(
        train_dataset=original_train_dataset,
        val_dataset=original_val_dataset,
        test_dataset=None,  # test 데이터셋이 없는 경우 None으로 설정
        output_dir=coco_data_org_root,
        image_format='jpg')
else :
    print(f"{coco_data_org_root} 폴더가 이미 존재합니다. COCO 변환을 건너뜁니다.")

In [ ]:
# cocco 데이타 셋 load
coco_data_org_root = f"{coco_data_root}_org"
print("coco_data_org_root:", coco_data_org_root)

train_coco_original = CocoDetection(
    root=f'{coco_data_org_root}/train2017',
    annFile=f'{coco_data_org_root}/instances_train2017.json'
)

val_coco_original = CocoDetection(
    root=f'{coco_data_org_root}/val2017',
    annFile=f'{coco_data_org_root}/instances_val2017.json'
)

print(len(train_coco_original), len(val_coco_original))


In [ ]:
def print_coco_class_distribution_df(coco_dataset, class_names=None, title="COCO 데이터셋 클래스 분포"):
    """
    COCO Detection 데이터셋의 클래스 분포를 DataFrame으로 출력 (가로 형태)
    """
    from collections import Counter

    class_counts = Counter()
    for _, targets in tqdm(coco_dataset, desc="클래스 분포 집계"):
        for ann in targets:
            class_id = ann.get('category_id', None)
            if class_id is not None:
                class_counts[class_id] += 1

    # 클래스명 매핑
    if class_names is None:
        # category_id가 0부터 시작한다고 가정
        class_names = [f"class_{i}" for i in range(max(class_counts.keys())+1)]

    # DataFrame 생성
    df = pd.DataFrame({
        '클래스명': [class_names[cid] for cid in sorted(class_counts.keys())],
        '샘플수': [class_counts[cid] for cid in sorted(class_counts.keys())],
        '비율(%)': [class_counts[cid] / sum(class_counts.values()) * 100 for cid in sorted(class_counts.keys())]
    })
    df = df.set_index('클래스명').T  # 전치하여 가로 형태로

    print("="*40)
    print(title)
    print("="*40)
    df.head_att(10)
    print(f"\n전체 샘플 수(객체 수): {sum(class_counts.values())}")
    print("="*40)
    return df

# 사용 예시
df_train_org = print_coco_class_distribution_df(train_coco_original, title="train_coco_original 클래스 분포")
df_val_org = print_coco_class_distribution_df(val_coco_original, title="val_coco_original 클래스 분포")

category_id_list=[]
class_names = list(df_train_org.columns)  # 또는 yaml에서 가져온 names
for class_id, class_name in enumerate(class_names):
    category_id = class_name.split('_')[1]
    category_id_list.append(category_id)
    print(f"category_id: {class_id}, class_name: {category_id}")

In [ ]:
def get_pill_optimized_augmentation():
    """
    알약 특성에 최적화된 증강 변환
    - 180도 회전 (알약이 굴러다니는 특성 반영)
    - ImageNet 평균값으로 배경 채움
    - 의료 이미지에 적합한 증강
    """
    
    # ImageNet 평균값을 0-255 범위로 변환
    imagenet_mean_rgb = [int(0.485 * 255), int(0.456 * 255), int(0.406 * 255)]  # [123, 116, 103]
    return transforms.Compose([
        transforms.RandomRotation(180, fill=imagenet_mean_rgb),  # 180도 회전 + ImageNet 평균값
        transforms.ColorJitter(
            brightness=0.3,
            contrast=0.3,
            saturation=0.2,
            hue=0.1
        ),
        transforms.RandomAffine(
            degrees=0,
            translate=(0.1, 0.1),
            scale=(0.9, 1.1),
            fill=imagenet_mean_rgb  # ImageNet 평균값으로 채움
        ),
        transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    ])
    

In [ ]:
def create_balanced_augmented_dataset(dataset, augmentation_fn, verbose=True, class_names=None):
    """
    클래스 분포의 최대값에 맞춰 증강하여 균일화된 데이터셋 생성
    Args:
        dataset: YOLOToClassificationDataset 또는 분류용 Dataset
        augmentation_fn: 증강 transform 함수 (예: get_pill_optimized_augmentation())
        verbose: 진행상황 출력 여부
    Returns:
        images: 증강된 이미지 리스트
        labels: 증강된 라벨 리스트
    """
    from collections import defaultdict
    class_indices = defaultdict(list)
    pbar = tqdm(range(len(dataset)), desc="클래스별 인덱스 그룹핑", **get_tqdm_kwargs())
    for idx in pbar:
        _, category_id = dataset[idx]  # label이 category_id임
        class_indices[category_id].append(idx)
    class_counts = {cid: len(idxs) for cid, idxs in class_indices.items()}
    max_count = max(class_counts.values())
    if verbose:
        print("클래스별 샘플 수:", class_counts)
        print("최대 샘플 수:", max_count)

    images, labels = [], []
    pbar = tqdm(class_indices.items(), desc="클래스별", **get_tqdm_kwargs())
    for category_id, idxs in pbar:
        for idx in idxs:
            img, label = dataset[idx]
            images.append(img)
            labels.append(label)
        n_to_add = max_count - len(idxs)
        if n_to_add > 0:
            for i in range(n_to_add):
                src_idx = idxs[i % len(idxs)]
                img, label = dataset[src_idx]
                aug_img = augmentation_fn(img)
                images.append(aug_img)
                labels.append(label)
        if verbose:
            name = class_names[category_id] if class_names is not None and category_id < len(class_names) else str(category_id)
            print(f"카테고리 {name}({category_id}): 증강 {n_to_add}개 추가")
    return images, labels

# 사용 예시
pill_aug_transform = get_pill_optimized_augmentation()
train_aug_images, train_aug_labels = create_balanced_augmented_dataset(
    original_train_dataset,
    augmentation_fn=pill_aug_transform,
    verbose=True,
    class_names=category_id_list  # names는 yaml에서 가져온 클래스명 리스트
)

val_aug_images, val_aug_labels = create_balanced_augmented_dataset(
    original_val_dataset,
    augmentation_fn=pill_aug_transform,
    verbose=True,
    class_names=category_id_list  # names는 yaml에서 가져온 클래스명 리스트
)

print(f"train 균일화된 증강 데이터셋 크기: {len(train_aug_images)}")
print(f"val 균일화된 증강 데이터셋 크기: {len(val_aug_images)}")

In [ ]:
coco_data_aug_root = f"{coco_data_root}_aug"
print("coco_data_aug_root:", coco_data_aug_root)

# 기존 폴더가 있으면 삭제 후 다시 생성
if os.path.exists(coco_data_aug_root):
    import shutil
    shutil.rmtree(coco_data_aug_root)

if not os.path.exists(coco_data_aug_root):
    os.makedirs(coco_data_aug_root)
    # COCO 형식으로 변환
    convert_yolo_to_coco_format(
        train_dataset=train_dataset,
        val_dataset=train_dataset,
        test_dataset=None,  # test 데이터셋이 없는 경우 None으로 설정
        output_dir=coco_data_aug_root,
        image_format='jpg')
else :
    print(f"{coco_data_aug_root} 폴더가 이미 존재합니다. COCO 변환을 건너뜁니다.")

